In [3]:
import openai
import pandas as pd
import seaborn as sns
import time
import os

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [19]:
## access the posts csvs
post_list_1 = pd.read_csv(r'data\refused_post_list.csv').Text.tolist()
post_list_2 = pd.read_csv(r'data\approved_post_list.csv').Text.tolist()
post_list_3 = pd.read_csv(r'data\refused_post_list.csv').Text.tolist()
post_list_4 = pd.read_csv(r'data\rejected_post_list.csv').Text.tolist()
post_list = post_list_1 + post_list_2 + post_list_3 + post_list_4

In [20]:
pd.DataFrame(post_list).to_csv(r"data\all_posts_new.csv")

In [21]:
### remove posts that are too short or too long

for post in post_list:
        if len(post) < 250 or len(post) > 5000:
            post_list.remove(post)

In [26]:
%%time
messages = []
responses = []
disclaimer = "One word answers only. Answer in the following format. Program: Degree Level: Univsersity: Sponsor: Scholarship: Relatives: Visa Attempt: Visa Result:"  
system_message = "You are a keyword extractor! Your job is to answer in the following question, strictly speaking, in only one word: What program is applicant applying for? What degree level is he applying or, bachelors, masters or phd? Which university is he applying visa to? Who is sponsoring the applicant? Did he recieve any scholarship? Did he had any relatives in US? How many visa attempts he has had(if not mentioned, it's first)? Was his visa approved or refused? If not applicable, say 'N/A' Program: Degree Level: University: Sponsor: Scholarship: Relatives: Visa Attempts: Visa Status:"
sample_question = "Visa interview Islamabad Texas A&M university 5th Aug: 11:30am Vo: Hello Me:hello sir  Vo: i need u to do your finger prints. (Messed up with left hand once) Me:done Vo: alright pass me those documents. Me: here Vo: so why are you going to the Us? Me: I'm going to pursue my bachelor's degree in health sciences at texas a&m university. Vo: how many universities did you apply to? Me: about 13 universities, I got accepted from a few. Vo: and which ones did u get accepted from? Me: i got accepted from the university of Hartford, texas a&m and university of---uh and I had a correspondence with the Baylor university. Vo: ok, so why did you choose this university? Me: well it has quite good ranking regarding my course, its listed as the 28th best medical university in terms of health care management and in biological scien--- Vo: alright so do you have any relatives in the Us? Me: Yes, an uncle of mine lives there with his family. Vo: where do they live? Me: Irving (accident said the old address) Vo: California?-- what state? Me: no not California, its in Dallas at Texas. Vo: alright and what does he do? Me: his work is associated with transport but I don't really know much about that. Vo: oh okay, so whose going to sponsor you? Me: my father will Vo: and what does he do? Me: told Vo: (going though my docs) have you ever been to the Us before? Me: yes, twice Vo: okay. (Keeps typing and looking through stuff for about a minute or two). Vo: alright, I'm approving your visa, but we'll be keeping your passport for further processing. You will receive it within 1-2weeks. There were some 30 candidates I saw that day and out of them only 2 girls & me myself got approved. One common thing between me and one of those girls was that we both had been to the Us before. Me twice for visiting and her for her bachelors(now going for masters I believe)."
sample_answer = "Program: Health Sciences Degree Level: Bachelors University: Texas A&M University Sponsor: Father Scholarship: N/A Relatives: Uncle Visa Attempts: First Visa Result: Approved"

for i in range(0, len(filtered_list)):
    time.sleep(3)
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo", # this is "ChatGPT" $0.002 per 1k tokens
      messages=[{"role": "system", "content": system_message},
                {"role": "user", "content": sample_question},
                {"role": "assistant", "content": sample_answer},
               {"role": "user", "content": disclaimer + filtered_list[i]}],
        temperature = 0
    )
    responses.append(completion.choices[0].message.content)

CPU times: total: 281 ms
Wall time: 31min 46s


In [27]:
print(f"The number of data we have is : {len(responses)}")

The number of data we have is : 373


In [28]:
str(responses[0].split(": ")[3]).replace("Sponsor", "").strip()
columns = ["Program", "Degree Level", "University", "Sponsor", "Scholarship", "Relatives", "Visa Attempt", "Visa Result", "Id"]

final_data = []
for i in range(len(responses)):
    values = str(responses[i]).split(": ")
    if len(values) != len(columns):
        print(f"Skipping Response Number: {i+1} *** Response: {responses[i][0:30]}...")
        continue
    data_dict = {}
    for j in range(0, len(columns)):
        key = columns[j-1]
        value = values[j].replace(columns[j], "").strip()
        data_dict[key] = value

    final_data.append(data_dict)
    

Skipping Response Number: 9 *** Response: Program: N/A
Degree Level: N/A...
Skipping Response Number: 23 *** Response: Program: MS Finance Degree Lev...
Skipping Response Number: 24 *** Response: Program: Civil Engineering Deg...
Skipping Response Number: 25 *** Response: Program: Environmental Health ...
Skipping Response Number: 58 *** Response: N/A...
Skipping Response Number: 66 *** Response: Program: Information Systems D...
Skipping Response Number: 75 *** Response: Program: Business Degree Level...
Skipping Response Number: 90 *** Response: N/A...
Skipping Response Number: 117 *** Response: Program: Medical Degree Level:...
Skipping Response Number: 125 *** Response: Program: Criminology Degree Le...
Skipping Response Number: 168 *** Response: Program: Business Analytics De...
Skipping Response Number: 169 *** Response: Program: PhD Degree Level: PhD...
Skipping Response Number: 170 *** Response: Program: MBA Degree Level: Mas...
Skipping Response Number: 173 *** Response: Progr

In [31]:
# Create a pandas DataFrame with the extracted data
df = pd.DataFrame(final_data)
df.head()

,Id,Program,Degree Level,University,Sponsor,Scholarship,Relatives,Visa Attempt,Visa Result
0,,Criminal Justice,N/A,Western Michigan University,Uncle,N/A,Uncle,Second,Refused
1,,N/A,N/A,N/A,N/A,N/A,N/A,Second (Refusal Category),N/A
2,,N/A,N/A,DePaul University,Father,Yes,N/A,Refused,N/A
3,,PhD in Physics,PhD,Purdue University,N/A,Fully funded,No,First,Refused
4,,Cyber Security,Masters,UD,Uncle,N/A,Uncle,First,Refused


In [30]:
### Saving the data
df.to_csv(r"data\training_data_new.csv")